In [ ]:
!nvidia-smi

Mon Oct 21 20:30:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   34C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install datasets transformers accelerate peft huggingface_hub hf_transfer trl wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.2/109.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
os.environ["HF_TOKEN"] = "X"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
os.environ["WANDB_API_KEY"] = "X"
os.environ["WANDB_PROJECT"] = "raft-Mixtral"
os.environ["WANDB_NAME"] = "RAFT-Mixtral"

In [ ]:
import torch
from datasets import load_dataset
from IPython.display import Markdown
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, EarlyStoppingCallback, TrainingArguments
from trl import SFTTrainer

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Is Bfloat16 avaiable?: {torch.cuda.is_bf16_supported()}")

Is Bfloat16 avaiable?: True


In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=model_name,
    padding_side="left"
)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
print(f"Vocabulary size of {model_name}: {len(tokenizer.get_vocab()):,}")

Vocabulary size of mistralai/Mistral-7B-Instruct-v0.2: 32,000


In [ ]:
tokenizer.special_tokens_map

{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}

In [ ]:
tokenizer.add_special_tokens({'pad_token': '<pad>'})
model.resize_token_embeddings(len(tokenizer))

Embedding(32001, 4096)

In [ ]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '<pad>'}

In [ ]:
generation_config = {
    "max_new_tokens": 500,
    "do_sample": True,
    "temperature": 0 + 1e-5,
    "top_k": 100,
    "top_p":0.90,
}

In [ ]:
chat = [
    {"role": "user", "content": "You are an ancient wizard, and you communicate with mystical riddles and enigmatic metaphors. What is the capital of France?"},
    {"role": "assistant", "content": "In the realm of light and culture, where the River Seine's gentle current flows, there lies a city of art and love. Its heart beats with the rhythm of creativity, pulsating in the bosom of the land of the Gauls. Seek the city where the grand Louvre stands guard, keeping the secrets of the Mona Lisa, and there you shall find the throne of the French realm. I speak, my dear interlocutor, of the City of Lights, the City of Love, the City of Paris"},
    {"role": "user", "content": "What is the capital of Germany?"}
]
formatted_chat = tokenizer.apply_chat_template(conversation=chat, tokenize=False, add_generation_prompt=True)
formatted_chat_token = tokenizer.apply_chat_template(conversation=chat, add_generation_prompt=True, return_tensors="pt")
print(formatted_chat)

<s> [INST] You are an ancient wizard, and you communicate with mystical riddles and enigmatic metaphors. What is the capital of France? [/INST] In the realm of light and culture, where the River Seine's gentle current flows, there lies a city of art and love. Its heart beats with the rhythm of creativity, pulsating in the bosom of the land of the Gauls. Seek the city where the grand Louvre stands guard, keeping the secrets of the Mona Lisa, and there you shall find the throne of the French realm. I speak, my dear interlocutor, of the City of Lights, the City of Love, the City of Paris</s> [INST] What is the capital of Germany? [/INST]


In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
print(model)

In [ ]:
input_ids = tokenizer(text=formatted_chat, return_tensors="pt").to(device)
print(input_ids)

{'input_ids': tensor([[    1,     1,   733, 16289, 28793,   995,   460,   396,  9467,   275,
         14640, 28725,   304,   368, 16287,   395, 11947,   745,   408,  2558,
           867,   304,   481,   326, 20095,  1424,  2366,   734, 28723,  1824,
           349,   272,  5565,   302,  4843, 28804,   733, 28748, 16289, 28793,
           560,   272, 18745,   302,  2061,   304,  5679, 28725,   970,   272,
          6396, 25517, 28742, 28713, 10434,  1868, 21142, 28725,   736, 10427,
           264,  2990,   302,  1524,   304,  2016, 28723,  7267,  3031,   347,
          1449,   395,   272, 19834,   302, 22317, 28725, 26691,  1077,   297,
           272, 16158,   300,   302,   272,  2533,   302,   272,   420,  2857,
         28713, 28723,  1091,   950,   272,  2990,   970,   272,  3487,  5105,
         19465, 10969,  6980, 28725,  7603,   272, 19376,   302,   272,  3217,
         28708, 18999, 28725,   304,   736,   368,  4579,  1300,   272, 24459,
           302,   272,  4949, 18745, 2

In [ ]:
outputs = model.generate(**input_ids, **generation_config)
print(outputs)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


tensor([[    1,     1,   733, 16289, 28793,   995,   460,   396,  9467,   275,
         14640, 28725,   304,   368, 16287,   395, 11947,   745,   408,  2558,
           867,   304,   481,   326, 20095,  1424,  2366,   734, 28723,  1824,
           349,   272,  5565,   302,  4843, 28804,   733, 28748, 16289, 28793,
           560,   272, 18745,   302,  2061,   304,  5679, 28725,   970,   272,
          6396, 25517, 28742, 28713, 10434,  1868, 21142, 28725,   736, 10427,
           264,  2990,   302,  1524,   304,  2016, 28723,  7267,  3031,   347,
          1449,   395,   272, 19834,   302, 22317, 28725, 26691,  1077,   297,
           272, 16158,   300,   302,   272,  2533,   302,   272,   420,  2857,
         28713, 28723,  1091,   950,   272,  2990,   970,   272,  3487,  5105,
         19465, 10969,  6980, 28725,  7603,   272, 19376,   302,   272,  3217,
         28708, 18999, 28725,   304,   736,   368,  4579,  1300,   272, 24459,
           302,   272,  4949, 18745, 28723,   315,  

In [ ]:
tokenizer.batch_decode(outputs[:, input_ids["input_ids"].shape[1]:])[0]

'In the heart of Europe, where the mighty rivers Rhine and Spree converge, lies a city of history and innovation. It is the cradle of great thinkers and artists, a beacon of culture and knowledge. Seek the city where the Brandenburg Gate stands tall, guarding the entrance to the land of poets and philosophers. I speak, my dear inquisitor, of the City of Spires, the City of Culture, the City of Berlin.</s>'

In [ ]:
#msg = tokenizer.decode(token_ids=outputs[0], skip_special_tokens=True)

In [ ]:
#model_inputs = formatted_chat_token.to(device)
#generated_ids = model.generate(model_inputs, **generation_config)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
#decoded = tokenizer.batch_decode(outputs)
#print(decoded[0])

<s><s> [INST] You are an ancient wizard, and you communicate with mystical riddles and enigmatic metaphors. What is the capital of France? [/INST] In the realm of light and culture, where the River Seine's gentle current flows, there lies a city of art and love. Its heart beats with the rhythm of creativity, pulsating in the bosom of the land of the Gauls. Seek the city where the grand Louvre stands guard, keeping the secrets of the Mona Lisa, and there you shall find the throne of the French realm. I speak, my dear interlocutor, of the City of Lights, the City of Love, the City of Paris</s> [INST] What is the capital of Germany? [/INST] In the heart of Europe, where the mighty rivers Rhine and Spree converge, lies a city of history and innovation. It is the cradle of great thinkers and artists, a beacon of culture and knowledge. Seek the city where the Brandenburg Gate stands tall, guarding the entrance to the land of poets and philosophers. I speak, my dear inquisitor, of the City of

In [ ]:
#print(msg)

[INST] You are an ancient wizard, and you communicate with mystical riddles and enigmatic metaphors. What is the capital of France? [/INST] In the realm of light and culture, where the River Seine's gentle current flows, there lies a city of art and love. Its heart beats with the rhythm of creativity, pulsating in the bosom of the land of the Gauls. Seek the city where the grand Louvre stands guard, keeping the secrets of the Mona Lisa, and there you shall find the throne of the French realm. I speak, my dear interlocutor, of the City of Lights, the City of Love, the City of Paris [INST] What is the capital of Germany? [/INST] In the heart of Europe, where the mighty rivers Rhine and Spree converge, lies a city of history and unity. It is the cradle of the great German nation, a beacon of culture and progress. Seek the city where the majestic Brandenburg Gate stands tall, guarding the entrance to the land of poets and thinkers. I speak, my dear inquisitor, of the City of Spires, the Ci

In [ ]:
dataset = load_dataset("DoganK01/RAFT-DATASET-HEALTHCARE", split="train")

README.md:   0%|          | 0.00/622 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/127M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3498 [00:00<?, ? examples/s]

In [ ]:
train_test_split = dataset.train_test_split(test_size=0.1, seed=1399, shuffle=True)
train_data = train_test_split["train"].shuffle()
val_data = train_test_split["test"].shuffle()
print(f"Train size: {len(train_data)}", f"Test size: {len(val_data)}")

Train size: 3148 Test size: 350


In [ ]:
torch.manual_seed(1399)
sample = train_data[torch.randint(low=0, high=len(train_data), size=(1,)).item()]

In [ ]:
def format_example(example, return_answer=True):

    instruction = "Carefully read and analyze the provided documents to answer the question that follows. " \
    "Provide a detailed, step-by-step explanation of your reasoning, demonstrating how you arrived at your" \
    " conclusion based on the information given in the documents.\n\n"

    chat = [{"role": "user", "content": instruction + example["instruction"]}]

    prompt = tokenizer.apply_chat_template(
        conversation=chat,
        tokenize=False,
        add_generation_prompt=True
    )

    if return_answer:
        prompt += f" {example['cot_answer']}</s>"

    return prompt

In [ ]:
print(format_example(example=sample, return_answer=True)[:1200])
print("...")
print(format_example(example=sample, return_answer=True)[-1200:])

<s> [INST] Carefully read and analyze the provided documents to answer the question that follows. Provide a detailed, step-by-step explanation of your reasoning, demonstrating how you arrived at your conclusion based on the information given in the documents.

<DOCUMENT>; Wang, C.Q.; Zhao, J. Complete genome sequence of a variant porcine epidemic diarrhea virus strain isolated in central china.Genome Announc.2013,1, e00243-12. [Google Scholar] [CrossRef] [Green Version]Song, D.; Chen, Y.; Peng, Q.; Huang, D.; Zhang, T.; Huang, T.; Zhang, F.; Zhou, X.; Tang, Y. Full-Length Genome Sequence of a Variant Porcine Epidemic Diarrhea Virus Strain, CH/GDZQ/2014, Responsible for a Severe Outbreak of Diarrhea in Piglets in Guangdong, China, 2014.Genome Announc.2014,2, e01239-14. [Google Scholar] [CrossRef]Murakami, S.; Miyazaki, A.; Takahashi, O.; Hashizume, W.; Hase, Y.; Ohashi, S.; Suzuki, T. Complete Genome Sequence of the Porcine Epidemic Diarrhea Virus Variant Tottori2/JPN/2014.Genome Announ

In [ ]:
"""
input_text = format_example(example=sample, return_answer=False)
input_ids = tokenizer(text=input_text, return_tensors="pt").to(device)
outputs = model.generate(**input_ids, **generation_config)
print(tokenizer.decode(token_ids=outputs[0], skip_special_tokens=True).split("GPT4 Correct Assistant:")[1])
"""

# **Baseline Performance**

In [ ]:
input_text = format_example(example=sample, return_answer=False)
input_ids = tokenizer(text=input_text, return_tensors="pt").to(device)
outputs = model.generate(**input_ids, **generation_config)
print(tokenizer.batch_decode(outputs[:, input_ids["input_ids"].shape[1]:])[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


The pulp of the carao has the highest protein content (16.56 µg/mg) among seeds, pulp, and crust.</s>


# **Parameter Efficient Fine-Tuning (PEFT)-LoRA**

In [ ]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32001, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
           

In [ ]:
model.gradient_checkpointing_enable()
if model.config.to_dict()["use_cache"]:
    model.use_cache = False

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
peft_model = get_peft_model(model=model, peft_config=peft_config)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print_trainable_parameters(peft_model)

trainable params: 41943040 || all params: 7283683328 || trainable%: 0.5758493074343608


In [ ]:
args_definition = dict(
    output_dir="/mistral-lora",
    overwrite_output_dir=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=3e-4,
    max_steps=500,
    lr_scheduler_type="cosine",
    max_grad_norm = 0.3,
    warmup_steps=100,
    logging_steps=20,
    save_steps=20,
    logging_first_step=True,
    seed=1399,
    bf16=True,
    report_to="wandb",
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    load_best_model_at_end=True
)
args = TrainingArguments(**args_definition)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
tokenizer.model_max_length

1000000000000000019884624838656

In [ ]:
trainer = SFTTrainer(
    model=peft_model,
    args=args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    peft_config=peft_config,
    formatting_func=format_example,
    max_seq_length=1024,
    packing=True,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, packing. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:195: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `S

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:421: UserWarning: You passed `packing=True` to the SFTTrainer/SFTConfig, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
20,1.782200,1.670036
40,1.621400,1.549406
60,1.533500,1.484815
80,1.467900,1.436862
100,1.434300,1.387532
120,1.355000,1.331776
140,1.333400,1.278482
160,1.257300,1.222494
180,1.187900,1.166641
200,1.167800,1.112111


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.

TrainOutput(global_step=500, training_loss=1.0428864600658416, metrics={'train_runtime': 64081.4569, 'train_samples_per_second': 0.25, 'train_steps_per_second': 0.008, 'total_flos': 7.03129901334528e+17, 'train_loss': 1.0428864600658416, 'epoch': 0.5534034311012729})

In [ ]:
fine_tuned_model = peft_model.merge_and_unload()

In [ ]:
input_text = format_example(example=sample, return_answer=False)
input_ids = tokenizer(text=input_text, return_tensors="pt").to(device)
outputs = fine_tuned_model.generate(**input_ids, **generation_config)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
print(tokenizer.batch_decode(outputs[:, input_ids["input_ids"].shape[1]:], skip_special_tokens=True)[0])

##Reason: The document provides information about the chemical composition of carao. It states ##begin_quote## Seeds, pulp, and crust have significantly different amounts of ash, lipids, proteins, and carbohydrate. ##end_quote## and further provides specific values for each component. The protein content is given as 16.56 ± 0.21% for the pulp. Therefore, the component with the highest protein content is the pulp. ##Answer: Pulp


In [ ]:
display(Markdown("#### Question:"))
display(Markdown(sample["question"]))
display(Markdown("#### RAFT FT Completion:"))
display(Markdown(tokenizer.batch_decode(outputs[:, input_ids["input_ids"].shape[1]:], skip_special_tokens=True)[0]))
display(Markdown("#### Answer:"))
display(Markdown(" " + sample["cot_answer"]))


#### Question:

Which carao component has the highest protein content?

#### RAFT FT Completion:

##Reason: The document provides information about the chemical composition of carao. It states ##begin_quote## Seeds, pulp, and crust have significantly different amounts of ash, lipids, proteins, and carbohydrate. ##end_quote## and further provides specific values for each component. The protein content is given as 16.56 ± 0.21% for the pulp. Therefore, the component with the highest protein content is the pulp. ##Answer: Pulp

#### Answer:

 ##Reason: The context provides data on the chemical composition of carao components such as seeds, pulp, and crust. It states ##begin_quote## The pulp of the carao has the highest moisture (12.55 ± 0.44%), ash (6.45 ± 0.15%), lipids (0.66 ± 0.07%), and proteins (16.56 ± 0.21%). ##end_quote## This indicates that the pulp has the highest protein content among the components. ##Answer: Pulp

In [ ]:
not_tuned_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

fine_tuned_model.use_cache = True

/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1390: UserWarning: Current model requires 4096 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def generate_responses(example, ft_model, og_model):
    input_text = format_example(example=example, return_answer=False)
    input_ids_ft = tokenizer(text=input_text, return_tensors="pt").to(device)
    #input_ids_og = tokenizer(text=input_text, return_tensors="pt")
    ft_outputs = ft_model.generate(**input_ids_ft, **generation_config)
    #og_outputs = og_model.generate(**input_ids_og, **generation_config)

    display(Markdown("#### Question:"))
    display(Markdown(example["question"]))
    #display(Markdown("#### Original Completion:"))
    #display(Markdown(tokenizer.batch_decode(og_outputs[:, input_ids_og["input_ids"].shape[1]:], skip_special_tokens=True)[0]))

    display(Markdown("#### RAFT Fine-tuned Completion:"))
    display(Markdown(tokenizer.batch_decode(ft_outputs[:, input_ids_ft["input_ids"].shape[1]:], skip_special_tokens=True)[0]))

    display(Markdown("#### Expected Answer:"))
    display(Markdown(" {answer}".format(answer=example["cot_answer"])))
    display(Markdown("-----------------------------"))

In [ ]:
for i in range(5):
    generate_responses(val_data[i], ft_model=fine_tuned_model, og_model=not_tuned_model)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


#### Question:

What are dark septate endophytes (DSEs)?

#### RAFT Fine-tuned Completion:

##Reason: The document describes dark septate endophytes (DSEs) as fungi that are conidial or sterile septate endophytes. They are commonly isolated from healthy plants and produce melanised structures, including inter- and intracellular hyphae and microsclerotia in the roots. DSEs have low host specificity and cover a wide geographical range. They are associated with various diseases of woody hosts, such as stem blight and canker, and dieback. ##Answer: Dark septate endophytes (DSEs) are fungi that are conidial or sterile septate endophytes, commonly isolated from healthy plants, which produce melanised structures and have low host specificity.

...

#### RAFT Fine-tuned Completion:

##Reason: The context provides information about the review of S. ciferrii complex infections. It states ##begin_quote## Twenty-two articles or trials were evaluated in this review. ##end_quote## This indicates the number of articles or trials that were included in the review. ##Answer: 22

#### Expected Answer:

 ##Reason: To determine how many articles were evaluated, we need to look at the process of selection in the review. The context mentions that ##begin_quote## Eighty-one articles or trials regarding S. ciferrii complex infections were identified. ##end_quote## After applying exclusion criteria, ##begin_quote## Forty-three articles were excluded. ##end_quote## Then, among the remaining ##begin_quote## thirty-eight articles or trials eligible for evaluation, ten were excluded as duplicated, while six were excluded after reading the abstract or full text. ##end_quote## This left ##begin_quote## Twenty-two articles or trials were evaluated in this review. ##end_quote## ##Answer: 22

-----------------------------

In [ ]:
model_save_name = "mistral-7b-instruct-raft-ft"

In [ ]:
# Save model & tokenizer
fine_tuned_model.push_to_hub(model_save_name)
tokenizer.push_to_hub(model_save_name)

  0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DoganK01/mistral-7b-instruct-raft-ft/commit/74d14eae364a7ca790a649bad34a99971d867757', commit_message='Upload tokenizer', commit_description='', oid='74d14eae364a7ca790a649bad34a99971d867757', pr_url=None, repo_url=RepoUrl('https://huggingface.co/DoganK01/mistral-7b-instruct-raft-ft', endpoint='https://huggingface.co', repo_type='model', repo_id='DoganK01/mistral-7b-instruct-raft-ft'), pr_revision=None, pr_num=None)

In [ ]:
# Save adapters
trainer.push_to_hub(model_save_name + "adapters")

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


  0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/524M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DoganK01/mistral-lora/commit/a2dffc85eaa324215c1980956fe127cf836e1dbc', commit_message='mistral-7b-instruct-raft-ftadapters', commit_description='', oid='a2dffc85eaa324215c1980956fe127cf836e1dbc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/DoganK01/mistral-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='DoganK01/mistral-lora'), pr_revision=None, pr_num=None)